In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *
import pyomo.environ as pyo
from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.util.calc_var_value import calculate_variable_from_constraint
from Method import Initialize
from Unit_Model import (H101_creater,R101_creater,F101_creater)
from pyomo.network import *

In [2]:
# Create a concrete model
m = ConcreteModel()

# Define a set for components
m.Phase = Set(initialize = ['Liq','Vap'])
m.Comp = Set(initialize = ['benzene','toluene','methane','hydrogen'])
m.R101 = R101_creater(m)
m.H101 = H101_creater(m)
m.F101 = F101_creater(m)

In [3]:
# 各组分进料
m.H101.inlet.flow['Liq','benzene'].fix(2.0008e-05)
m.H101.inlet.flow['Liq','toluene'].fix(0.30001)
m.H101.inlet.flow['Liq','methane'].fix(2.0008e-05)
m.H101.inlet.flow['Liq','hydrogen'].fix(2.0008e-05)
m.H101.inlet.flow['Vap','benzene'].fix(0.34136)
m.H101.inlet.flow['Vap','toluene'].fix(0.033075)
m.H101.inlet.flow['Vap','methane'].fix(1.0994)
m.H101.inlet.flow['Vap','hydrogen'].fix(0.50088)

# 温度压力
m.H101.inlet.P.fix(350000)
m.H101.inlet.T.fix(330.80)

m.H101.outlet.P.fix(350000)
m.H101.outlet.T.fix(600)

m.F101.outlet.P.fix(350000)
m.F101.outlet.T.fix(325)

m.R101.Q.fix(0)

Initialize(m.H101)

In [4]:
solver = SolverFactory('ipopt')  # Choose an appropriate solver
solver.solve(m.H101)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 29, 'Number of variables': 29, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.09887194633483887}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [5]:
m.H101.pprint()

H101 : Size=1, Index=None, Active=True
    7 Set Declarations
        Comp : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    4 : {'benzene', 'toluene', 'methane', 'hydrogen'}
        Phase : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Liq', 'Vap'}
        cp_ig_1_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : H101.Phase*H101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), ('Liq', 'hydrogen'), ('Vap', 'benzene'), ('Vap', 'toluene'), ('Vap', 'methane'), ('Vap', 'hydrogen')}
        cp_ig_2_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : H101.Phase*H101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), 

In [8]:
m.H101.Outlet.flow['Vap','benzene'].value

0.341380008

In [9]:
m.R101.inlet.flow['Vap','benzene'].value

1.0359973075428295

In [6]:
m.s1 = Arc(source=m.H101.Outlet, destination=m.R101.Inlet)
m.s2 = Arc(source=m.R101.Outlet, destination=m.F101.Inlet)

TransformationFactory("network.expand_arcs").apply_to(m)
#solver.solve(m.R101)
#m.R101.pprint()

In [8]:
m.s2 = Arc(source=m.R101.Outlet, destination=m.F101.Inlet)
TransformationFactory("network.expand_arcs").apply_to(m)


'pyomo.network.arc.ScalarArc'>) on block unknown with a new Component
(type=<class 'pyomo.network.arc.ScalarArc'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().


AttributeError: 'NoneType' object has no attribute 'component'

In [5]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 3

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

In [6]:
guesses_H101 = {
    "flow": {
        ("Vap", "benzene"): 1e-5,
        ("Vap", "toluene"): 1e-5,
        ("Vap", "hydrogen"): 1e-5,
        ("Vap", "methane"): 1e-5,
        ("Liq", "benzene"): 0.34138,
        ("Liq", "toluene"): 0.3331,
        ("Liq", "hydrogen"): 0.5009,
        ("Liq", "methane"): 1.0994,
    },
    "T": {0: 600},
    "P": {0: 350000},
}
seq.set_guesses_for(m.H101.outlet, guesses_H101)

guesses_R101 = {
    "flow": {
        ("Vap", "benzene"): 1e-5,
        ("Vap", "toluene"): 1e-5,
        ("Vap", "hydrogen"): 1e-5,
        ("Vap", "methane"): 1e-5,
        ("Liq", "benzene"): 0.208,
        ("Liq", "toluene"): 0.08327,
        ("Liq", "hydrogen"): 0.25109,
        ("Liq", "methane"): 1.34921,
    },
    "T": {0: 745.24},
    "P": {0: 350000},
}
seq.set_guesses_for(m.R101.outlet, guesses_R101)

guesses_F101 = {
    "flow": {
        ("Vap", "benzene"): 0.1457,
        ("Vap", "toluene"): 0.01512,
        ("Vap", "hydrogen"): 0.3272,
        ("Vap", "methane"): 1.26767,
        ("Liq", "benzene"): 0.59119,
        ("Liq", "toluene"): 0.06301,
        ("Liq", "hydrogen"): 0.001008,
        ("Liq", "methane"): 0.00443,
    },
    "T": {0: 325},
    "P": {0: 350000},
}
seq.set_guesses_for(m.F101.outlet, guesses_R101)

In [7]:
# Initialize the model using Sequential Decomposition
#seq.run(m)

# Solve the initialized model
solver = SolverFactory('ipopt')  # Choose an appropriate solver
results = solver.solve(m, tee=True)

# Check if the initialization was successful
if results.solver.termination_condition == TerminationCondition.optimal:
    print("Initialization successful.")
else:
    print("Initialization failed. Check the model and constraints.")

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [10]:
# 模型求解
solver = pyo.SolverFactory('ipopt')
solver.options['tol'] = 1e-6
solver.options['max_iter'] = 6000
results = solver.solve(m,tee=True)

Ipopt 3.13.2: tol=1e-06
max_iter=6000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        

In [8]:
m.F101.pprint()

F101 : Size=1, Index=None, Active=True
    7 Set Declarations
        Comp : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    4 : {'benzene', 'toluene', 'methane', 'hydrogen'}
        Phase : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Liq', 'Vap'}
        cp_ig_1_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : F101.Phase*F101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), ('Liq', 'hydrogen'), ('Vap', 'benzene'), ('Vap', 'toluene'), ('Vap', 'methane'), ('Vap', 'hydrogen')}
        cp_ig_2_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : F101.Phase*F101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), 